In [1]:
"""
PyTorch script for model training (MobileNetV2).
Copyright (C) 2021 by Akira TAMAMORI
Permission is hereby granted, free of charge, to any person obtaining
a copy of this software and associated documentation files (the
"Software"), to deal in the Software without restriction, including
without limitation the rights to use, copy, modify, merge, publish,
distribute, sublicense, and/or sell copies of the Software, and to
permit persons to whom the Software is furnished to do so, subject to
the following conditions:
The above copyright notice and this permission notice shall be
included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE
LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION
WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""

# Standard library imports.
import datetime
import os
import sys
from itertools import chain
import random
# Related third party imports.
import joblib
import numpy
import scipy.stats
import torch
import torch.utils.data
from scipy.special import softmax
from torch import optim
from torchinfo import summary
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

# Local application/library specific imports.
import util
from pytorch_model import Transformer_Decoder as Model

# Load configuration from YAML file.
CONFIG = util.load_yaml("./config.yaml")

# String constant: "cuda:0" or "cpu"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def set_seed(seed: int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    numpy.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(42)

In [3]:
def flatten(nested_list):
    """
    Flatten list.
    """
    return list(chain.from_iterable(nested_list))


def concat_features(file_list):
    """
    Extract features from audio files and then concate them into an array.
    """
    # calculate the number of dimensions
    n_features = []
    for file_id, file_name in enumerate(file_list):

        # extract feature from audio file.
        feature = util.extract_feature(file_name, CONFIG["feature"])
        feature = feature[:: CONFIG["feature"]["n_hop_frames"], :]

        if file_id == 0:
            features = numpy.zeros(
                (
                    len(file_list) * feature.shape[0],
                    CONFIG["feature"]["n_mels"] * CONFIG["feature"]["n_frames"],
                ),
                dtype=float,
            )

        features[
            feature.shape[0] * file_id : feature.shape[0] * (file_id + 1), :
        ] = feature

        n_features.append(feature.shape[0])

    return features, n_features


class DcaseDataset(torch.utils.data.Dataset):
    """
    Prepare dataset.
    """

    def __init__(self, unique_section_names, target_dir, mode):
        super().__init__()

        n_files_ea_section = []  # number of files for each section
        n_vectors_ea_file = []  # number of vectors for each file
        data = numpy.empty(
            (0, CONFIG["feature"]["n_frames"] * CONFIG["feature"]["n_mels"]),
            dtype=float,
        )
        
        for section_name in unique_section_names:
            # get file list for each section
            # all values of y_true are zero in training
            print("target_dir : %s" % (target_dir + "_" + section_name))
            files, _ = util.file_list_generator(
                target_dir=target_dir,
                section_name=section_name,
                dir_name="train",
                mode=mode,
            )
            print("number of files : %s" % (str(len(files))))

            n_files_ea_section.append(len(files))

            # extract features from audio files and
            # concatenate them into Numpy array.
            features, n_features = concat_features(files)

            data = numpy.append(data, features, axis=0)
            n_vectors_ea_file.append(n_features)

        n_vectors_ea_file = flatten(n_vectors_ea_file)

        # make target labels for conditioning
        # they are not one-hot vector!
        labels = numpy.zeros((data.shape[0]), dtype=int)
        start_index = 0
        for section_index in range(unique_section_names.shape[0]):
            for file_id in range(n_files_ea_section[section_index]):
                labels[
                    start_index : start_index + n_vectors_ea_file[file_id]
                ] = section_index
                start_index += n_vectors_ea_file[file_id]

        # 1D vector to 2D image (1ch)
        self.data = data.reshape(
            (
                data.shape[0],
                #1,  # number of channels
                CONFIG["feature"]["n_frames"],
                CONFIG["feature"]["n_mels"],
            )
        )

        self.labels = labels

    def __len__(self):
        return self.data.shape[0]  # return num of samples

    def __getitem__(self, index):
        sample = self.data[index, :]
        label = self.labels[index]

        return sample, label


In [4]:
def get_dataloader(dataset):
    """
    Make dataloader from dataset for training.
    """
    train_size = int(len(dataset) * (1.0 - CONFIG["training"]["validation_split"]))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size])

    data_loader_train = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=CONFIG["training"]["batch_size"],
        shuffle=CONFIG["training"]["shuffle"],
        drop_last=True,
    )
    data_loader_val = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=CONFIG["training"]["batch_size"],
        shuffle=False,
        drop_last=False,
    )

    # dataloader of training data for evaluation only
    data_loader_eval_train = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=CONFIG["training"]["batch_size"],
        shuffle=False,
        drop_last=False,
    )

    return data_loader_train, data_loader_val, data_loader_eval_train


def get_model(embed_dim, seq_len):
    """
    Instantiate AutoEncoder.
    """
    model = Model(embed_dim, seq_len-1).to(DEVICE)
    return model


def get_optimizer(model):
    """
    Instantiate optimizer.
    """

    optimizer = optim.Adam(
        params=model.parameters(),
        weight_decay=CONFIG["training"]["weight_decay"],
        lr=CONFIG["training"]["learning_rate"],
    )

    # optional
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=CONFIG["training"]["lr_step_size"],
        gamma=CONFIG["training"]["lr_gamma"],
    )

    return optimizer, scheduler

In [5]:
def calc_anomaly_score(model, file_path, section_index):
    """
    Calculate anomaly score.
    """
    try:
        # extract features (log-mel spectrogram)
        data = util.extract_feature(file_name=file_path, config=CONFIG["feature"])
        data = data.reshape(
            (  # must be a tuple of ints
                data.shape[0],
                1,
                CONFIG["feature"]["n_frames"],
                CONFIG["feature"]["n_mels"],
            )
        )
    except FileNotFoundError:
        print("File broken!!: {}".format(file_path))

    condition = numpy.zeros((data.shape[0]), dtype=int)
    if section_index != -1:
        condition[:] = section_index

    feed_data = torch.from_numpy(data).clone()
    feed_data = feed_data.to(DEVICE).float()
    with torch.no_grad():
        output = model(feed_data)  # notice: unnormalized output
        output = output.to("cpu").detach().numpy().copy()  # tensor to numpy array.

    output = softmax(output, axis=1)
    prob = output[:, section_index]

    y_pred = numpy.mean(
        numpy.log(
            numpy.maximum(1.0 - prob, sys.float_info.epsilon)
            - numpy.log(numpy.maximum(prob, sys.float_info.epsilon))
        )
    )

    return y_pred


def fit_gamma_dist(model, target_dir, mode):
    """
    - Calculate anomaly scores over sections.
    - Fit gamma distribution for anomaly scores.
    - Save the parameters of the distribution.
    """

    section_names = util.get_section_names(target_dir, dir_name="train")
    dataset_scores = numpy.array([], dtype=numpy.float64)

    # calculate anomaly scores over sections
    for section_index, section_name in enumerate(section_names):
        section_files, _ = util.file_list_generator(
            target_dir=target_dir,
            section_name=section_name,
            dir_name="train",
            mode=mode,
        )
        section_scores = [0.0 for k in section_files]
        for file_idx, file_path in enumerate(section_files):
            section_scores[file_idx] = calc_anomaly_score(
                model, file_path=file_path, section_index=section_index
            )

        section_scores = numpy.array(section_scores)
        dataset_scores = numpy.append(dataset_scores, section_scores)

    dataset_scores = numpy.array(dataset_scores)

    # fit gamma distribution for anomaly scores
    gamma_params = scipy.stats.gamma.fit(dataset_scores)
    gamma_params = list(gamma_params)

    # save the parameters of the distribution
    score_file_path = "{model}/score_distr_{machine_type}.pkl".format(
        model=CONFIG["model_directory"], machine_type=os.path.split(target_dir)[1]
    )
    joblib.dump(gamma_params, score_file_path)


def save_model(model, model_dir, machine_type):
    """
    Save PyTorch model.
    """

    model_file_path = "{model}/model_{machine_type}.hdf5".format(
        model=model_dir, machine_type=machine_type
    )
    # if os.path.exists(model_file_path):
    #     print("Model already exists!")
    #     continue
    torch.save(model.state_dict(), model_file_path)
    print("save_model -> %s" % (model_file_path))

In [6]:
def training(model, data_loader, optimizer, scheduler=None):
    """
    Perform training
    """
    model.train()  # training mode
    train_loss = 0.0
    for data, label in data_loader:
        data = data.to(DEVICE).float()
        label = label.to(DEVICE).long()
        optimizer.zero_grad()  # reset gradient
        loss = model.get_loss(data, label)
        loss.backward()  # backpropagation
        train_loss += loss.item()
        optimizer.step()  # update paramerters

    if scheduler is not None:
        scheduler.step()  # update learning rate
    #_ = model.get_loss(data, label, imshow=True)
    # model.eval()
    # correct = 0
    # total = 0
    # with torch.no_grad():
    #     for data, label in data_loader:
    #         data = data.to(DEVICE).float()
    #         label = label.to(DEVICE).long()
    #         outputs = model.get_loss(data)
    #         _, predicted = torch.max(outputs, 1)
    #         correct += (predicted == label).sum().item()
    #         total += label.size(0)

    print("loss: {:.6f} - ".format(train_loss / len(data_loader)))
    # print(
    #     "accuracy: {:.6f}% ({}/{})".format(
    #         100 * float(correct / total), correct, total
    #     ),
    # )


def validation(model, data_loader):
    """
    Perform validation
    """
    model.eval()  # validation mode
    val_loss = 0.0
    with torch.no_grad():
        for data, label in data_loader:
            data = data.to(DEVICE).float()
            label = label.to(DEVICE).long()
            loss = model.get_loss(data, label)
            loss = loss.mean()
            val_loss += loss.item()
        #_ = model.get_loss(data, label, imshow=True)
            #outputs = model(data)
            # _, predicted = torch.max(outputs, 1)
            # correct += (predicted == label).sum().item()
            # total += label.size(0)

    print("loss: {:.6f} - ".format(val_loss / len(data_loader)))
    # print(
    #     "accuracy: {:.6f}% ({}/{})".format(
    #         100 * float(correct / total), correct, total
    #     ),
    # )

In [7]:
def main(mode):
    """
    Perform model training and validation.
    """

    # check mode
    # "development": mode == True
    # "evaluation": mode == False
    #mode = util.command_line_chk()  # constant: True or False
    #if mode is None:
    #    sys.exit(-1)

    # make output directory
    os.makedirs(CONFIG["model_directory"], exist_ok=True)

    # load base_directory list
    dir_list = util.select_dirs(config=CONFIG, mode=mode)

    for idx, target_dir in enumerate(dir_list):
        print("===============================================")
        print("[%d/%d] %s" % (idx + 1, len(dir_list), target_dir))

        section_names_file_path = "{model}/section_names_{machine_type}.pkl".format(
            model=CONFIG["model_directory"], machine_type=os.path.split(target_dir)[1]
        )
        unique_section_names = numpy.unique(
            util.get_section_names(target_dir, dir_name="train")
        )
        joblib.dump(unique_section_names, section_names_file_path)

        print("\n============== DATASET_GENERATOR ==============")
        # for debug ########################################
        #unique_section_names = unique_section_names[0:1]
        ####################################################
        dcase_dataset = DcaseDataset(unique_section_names, target_dir, mode)
        print("===============================================")

        print("\n=========== DATALOADER_GENERATOR ==============")
        data_loader = {"train": None, "val": None, "eval_train": None}
        (
            data_loader["train"],
            data_loader["val"],
            data_loader["eval_train"],
        ) = get_dataloader(dcase_dataset)
        print("===============================================")

        print("\n================ MODEL TRAINING ===============")
        model = get_model(embed_dim=CONFIG['feature']['n_mels'], seq_len=CONFIG['feature']['n_frames'])
        optimizer, _ = get_optimizer(model)
        # optimizer, scheduler = get_optimizer(model)  # optional

        # display summary of model through torchinfo
        #summary(
        #    model,
        #    input_size=(
        #        CONFIG["training"]["batch_size"],
        #        1,  # number of channels
        #        CONFIG["feature"]["n_frames"],
        #        CONFIG["feature"]["n_mels"],
        #    ),
        #)

        # training loop
        for epoch in range(1, CONFIG["training"]["epochs"] + 1):
            now = datetime.datetime.now()
            now_str = now.strftime("%Y/%m/%d %H:%M:%S")
            print("{} Epoch {:2d} Train: ".format(now_str, epoch), end="")
            training(
                model=model,
                data_loader=data_loader["train"],
                optimizer=optimizer,
                # scheduler=scheduler  # optional
            )
            #now = datetime.datetime.now()
            #now_str = now.strftime("%Y/%m/%d %H:%M:%S")
            print("{} Epoch {:2d} Valid: ".format(now_str, epoch), end="")
            validation(model=model, data_loader=data_loader["val"])

        del dcase_dataset, data_loader

        # fit gamma distribution for anomaly scores
        # and save the parameters of the distribution
        #fit_gamma_dist(model=model, target_dir=target_dir, mode=mode)

        print("============== SAVE MODEL ==============")
        save_model(
            model,
            model_dir=CONFIG["model_directory"],
            machine_type=os.path.split(target_dir)[1],
        )

        print("============== END TRAINING ==============")
#if __name__ == "__main__":
#    main()

In [8]:
main(mode=True)

load_directory <- development
[1/7] /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/ToyCar

============== DATASET_GENERATOR ==============
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/ToyCar_section_00
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/ToyCar_section_01
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/ToyCar_section_02
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/ToyCar_section_03
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/ToyCar_section_04
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/ToyCar_section_05
number of files : 1003

=========== DATALOADER_GENERATOR ==============

================ MODEL TRAINING ===============
2021/05/08

2021/05/08 02:08:27 Epoch 65 Train: loss: 11.308329 - 
2021/05/08 02:08:27 Epoch 65 Valid: loss: 16.480862 - 
2021/05/08 02:08:47 Epoch 66 Train: loss: 11.296405 - 
2021/05/08 02:08:47 Epoch 66 Valid: loss: 16.793276 - 
2021/05/08 02:09:07 Epoch 67 Train: loss: 11.283757 - 
2021/05/08 02:09:07 Epoch 67 Valid: loss: 16.451652 - 
2021/05/08 02:09:27 Epoch 68 Train: loss: 11.271393 - 
2021/05/08 02:09:27 Epoch 68 Valid: loss: 16.262731 - 
2021/05/08 02:09:47 Epoch 69 Train: loss: 11.258615 - 
2021/05/08 02:09:47 Epoch 69 Valid: loss: 16.601642 - 
2021/05/08 02:10:06 Epoch 70 Train: loss: 11.247202 - 
2021/05/08 02:10:06 Epoch 70 Valid: loss: 16.342701 - 
2021/05/08 02:10:26 Epoch 71 Train: loss: 11.237184 - 
2021/05/08 02:10:26 Epoch 71 Valid: loss: 16.420196 - 
2021/05/08 02:10:46 Epoch 72 Train: loss: 11.224533 - 
2021/05/08 02:10:46 Epoch 72 Valid: loss: 16.500482 - 
2021/05/08 02:11:06 Epoch 73 Train: loss: 11.213288 - 
2021/05/08 02:11:06 Epoch 73 Valid: loss: 16.295365 - 
2021/05/08

2021/05/08 02:29:55 Epoch 27 Train: loss: 21.457056 - 
2021/05/08 02:29:55 Epoch 27 Valid: loss: 62.884073 - 
2021/05/08 02:30:14 Epoch 28 Train: loss: 21.101012 - 
2021/05/08 02:30:14 Epoch 28 Valid: loss: 63.895475 - 
2021/05/08 02:30:33 Epoch 29 Train: loss: 20.822623 - 
2021/05/08 02:30:33 Epoch 29 Valid: loss: 62.999237 - 
2021/05/08 02:30:53 Epoch 30 Train: loss: 20.583405 - 
2021/05/08 02:30:53 Epoch 30 Valid: loss: 64.852832 - 
2021/05/08 02:31:12 Epoch 31 Train: loss: 20.388619 - 
2021/05/08 02:31:12 Epoch 31 Valid: loss: 64.225855 - 
2021/05/08 02:31:31 Epoch 32 Train: loss: 20.188014 - 
2021/05/08 02:31:31 Epoch 32 Valid: loss: 65.315022 - 
2021/05/08 02:31:51 Epoch 33 Train: loss: 20.015036 - 
2021/05/08 02:31:51 Epoch 33 Valid: loss: 63.511054 - 
2021/05/08 02:32:10 Epoch 34 Train: loss: 19.831123 - 
2021/05/08 02:32:10 Epoch 34 Valid: loss: 64.653701 - 
2021/05/08 02:32:29 Epoch 35 Train: loss: 19.675374 - 
2021/05/08 02:32:29 Epoch 35 Valid: loss: 65.164788 - 
2021/05/08

target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/fan_section_01
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/fan_section_02
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/fan_section_03
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/fan_section_04
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/fan_section_05
number of files : 1003

=========== DATALOADER_GENERATOR ==============

================ MODEL TRAINING ===============
2021/05/08 02:55:09 Epoch  1 Train: loss: 126.423350 - 
2021/05/08 02:55:09 Epoch  1 Valid: loss: 104.962660 - 
2021/05/08 02:55:28 Epoch  2 Train: loss: 77.013338 - 
2021/05/08 02:55:28 Epoch  2 Valid: loss: 47.677289 - 
2021/05/08 02:55:47 Epoch  3 Train: loss: 33.782213 - 
2021/05/08 02:55:47 Epoch  3 V

2021/05/08 03:16:39 Epoch 68 Train: loss: 11.089436 - 
2021/05/08 03:16:39 Epoch 68 Valid: loss: 13.114980 - 
2021/05/08 03:16:58 Epoch 69 Train: loss: 11.074962 - 
2021/05/08 03:16:58 Epoch 69 Valid: loss: 13.006904 - 
2021/05/08 03:17:17 Epoch 70 Train: loss: 11.063120 - 
2021/05/08 03:17:17 Epoch 70 Valid: loss: 12.925922 - 
2021/05/08 03:17:37 Epoch 71 Train: loss: 11.054442 - 
2021/05/08 03:17:37 Epoch 71 Valid: loss: 13.018281 - 
2021/05/08 03:17:56 Epoch 72 Train: loss: 11.041094 - 
2021/05/08 03:17:56 Epoch 72 Valid: loss: 12.902784 - 
2021/05/08 03:18:15 Epoch 73 Train: loss: 11.031468 - 
2021/05/08 03:18:15 Epoch 73 Valid: loss: 13.024612 - 
2021/05/08 03:18:35 Epoch 74 Train: loss: 11.021820 - 
2021/05/08 03:18:35 Epoch 74 Valid: loss: 12.965813 - 
2021/05/08 03:18:54 Epoch 75 Train: loss: 11.010820 - 
2021/05/08 03:18:54 Epoch 75 Valid: loss: 13.030477 - 
2021/05/08 03:19:13 Epoch 76 Train: loss: 11.002666 - 
2021/05/08 03:19:13 Epoch 76 Valid: loss: 13.082142 - 
2021/05/08

...

target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/pump_section_01
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/pump_section_02
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/pump_section_03
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/pump_section_04
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/pump_section_05
number of files : 1003

=========== DATALOADER_GENERATOR ==============

================ MODEL TRAINING ===============
2021/05/08 04:03:01 Epoch  1 Train: loss: 139.850622 - 
2021/05/08 04:03:01 Epoch  1 Valid: loss: 119.140838 - 
2021/05/08 04:03:20 Epoch  2 Train: loss: 89.546494 - 
2021/05/08 04:03:20 Epoch  2 Valid: loss: 58.939145 - 
2021/05/08 04:03:39 Epoch  3 Train: loss: 42.664980 - 
2021/05/08 04:03:39 Epoch

2021/05/08 04:24:27 Epoch 68 Train: loss: 11.604240 - 
2021/05/08 04:24:27 Epoch 68 Valid: loss: 15.545359 - 
2021/05/08 04:24:46 Epoch 69 Train: loss: 11.588305 - 
2021/05/08 04:24:46 Epoch 69 Valid: loss: 15.498084 - 
2021/05/08 04:25:05 Epoch 70 Train: loss: 11.576992 - 
2021/05/08 04:25:05 Epoch 70 Valid: loss: 15.740297 - 
2021/05/08 04:25:24 Epoch 71 Train: loss: 11.559787 - 
2021/05/08 04:25:24 Epoch 71 Valid: loss: 15.091813 - 
2021/05/08 04:25:43 Epoch 72 Train: loss: 11.545402 - 
2021/05/08 04:25:43 Epoch 72 Valid: loss: 15.707513 - 
2021/05/08 04:26:02 Epoch 73 Train: loss: 11.528099 - 
2021/05/08 04:26:02 Epoch 73 Valid: loss: 15.777522 - 
2021/05/08 04:26:22 Epoch 74 Train: loss: 11.521837 - 
2021/05/08 04:26:22 Epoch 74 Valid: loss: 15.464762 - 
2021/05/08 04:26:41 Epoch 75 Train: loss: 11.503332 - 
2021/05/08 04:26:41 Epoch 75 Valid: loss: 15.732617 - 
2021/05/08 04:27:00 Epoch 76 Train: loss: 11.494355 - 
2021/05/08 04:27:00 Epoch 76 Valid: loss: 15.765580 - 
2021/05/08

2021/05/08 04:45:42 Epoch 30 Valid: loss: 18.347777 - 
2021/05/08 04:46:01 Epoch 31 Train: loss: 13.728525 - 
2021/05/08 04:46:01 Epoch 31 Valid: loss: 18.314749 - 
2021/05/08 04:46:21 Epoch 32 Train: loss: 13.670224 - 
2021/05/08 04:46:21 Epoch 32 Valid: loss: 18.557937 - 
2021/05/08 04:46:40 Epoch 33 Train: loss: 13.618578 - 
2021/05/08 04:46:40 Epoch 33 Valid: loss: 18.655749 - 
2021/05/08 04:46:59 Epoch 34 Train: loss: 13.566930 - 
2021/05/08 04:46:59 Epoch 34 Valid: loss: 18.309942 - 
2021/05/08 04:47:18 Epoch 35 Train: loss: 13.513291 - 
2021/05/08 04:47:18 Epoch 35 Valid: loss: 18.219399 - 
2021/05/08 04:47:38 Epoch 36 Train: loss: 13.470300 - 
2021/05/08 04:47:38 Epoch 36 Valid: loss: 18.200805 - 
2021/05/08 04:47:57 Epoch 37 Train: loss: 13.424383 - 
2021/05/08 04:47:57 Epoch 37 Valid: loss: 18.134272 - 
2021/05/08 04:48:16 Epoch 38 Train: loss: 13.383716 - 
2021/05/08 04:48:16 Epoch 38 Valid: loss: 18.430663 - 
2021/05/08 04:48:35 Epoch 39 Train: loss: 13.342132 - 
2021/05/08

target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/valve_section_01
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/valve_section_02
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/valve_section_03
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/valve_section_04
number of files : 1003
target_dir : /media/hiroki/HDD1TB/research/dcase2021_task2/datasets_add/dev_data/valve_section_05
number of files : 1003

=========== DATALOADER_GENERATOR ==============

================ MODEL TRAINING ===============
2021/05/08 05:09:59 Epoch  1 Train: loss: 143.107033 - 
2021/05/08 05:09:59 Epoch  1 Valid: loss: 117.553967 - 
2021/05/08 05:10:18 Epoch  2 Train: loss: 84.763879 - 
2021/05/08 05:10:18 Epoch  2 Valid: loss: 51.455816 - 
2021/05/08 05:10:37 Epoch  3 Train: loss: 38.016366 - 
2021/05/08 05:10:37 

2021/05/08 05:31:18 Epoch 68 Train: loss: 12.847164 - 
2021/05/08 05:31:18 Epoch 68 Valid: loss: 20.625361 - 
2021/05/08 05:31:37 Epoch 69 Train: loss: 12.826635 - 
2021/05/08 05:31:37 Epoch 69 Valid: loss: 20.415528 - 
2021/05/08 05:31:56 Epoch 70 Train: loss: 12.811259 - 
2021/05/08 05:31:56 Epoch 70 Valid: loss: 20.164705 - 
2021/05/08 05:32:15 Epoch 71 Train: loss: 12.798055 - 
2021/05/08 05:32:15 Epoch 71 Valid: loss: 20.101109 - 
2021/05/08 05:32:34 Epoch 72 Train: loss: 12.780507 - 
2021/05/08 05:32:34 Epoch 72 Valid: loss: 20.285206 - 
2021/05/08 05:32:53 Epoch 73 Train: loss: 12.766935 - 
2021/05/08 05:32:53 Epoch 73 Valid: loss: 20.577981 - 
2021/05/08 05:33:12 Epoch 74 Train: loss: 12.755463 - 
2021/05/08 05:33:12 Epoch 74 Valid: loss: 19.997154 - 
2021/05/08 05:33:31 Epoch 75 Train: loss: 12.739743 - 
2021/05/08 05:33:31 Epoch 75 Valid: loss: 20.275541 - 
2021/05/08 05:33:50 Epoch 76 Train: loss: 12.724361 - 
2021/05/08 05:33:50 Epoch 76 Valid: loss: 20.521072 - 
2021/05/08